In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from os import *
from pathlib import *
import numpy as np
from PIL import Image

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
__file__ = Path(getcwd())

In [72]:
root = path.join(__file__.parent, 'data')
images_path = path.join(root, "PNGImages")
masks_path = path.join(root, "PedMasks")
images = listdir(images_path)
masks = listdir(masks_path)

def get(idx):
    image_path = path.join(images_path, images[idx])
    mask_path = path.join(masks_path, masks[idx])

    image = Image.open(image_path).convert("RGB")
    mask = np.array(Image.open(mask_path))

    obj_ids = np.unique(mask)

    return mask, obj_ids

mask, ids = get(0)
mask = mask == mask[:, None, None]
mask.shape

(536, 1, 536, 559)

In [ ]:
class PennFudanDataset(data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transform = transforms

        self.images_path = path.join(self.root, "PNGImages")
        self.images = listdir(self.images_path)

        self.masks_path = path.join(self.root, "PedMasks")
        self.masks = listdir(self.masks_path)

    def __getitem__(self, idx):
        img_path = path.join(self.images_path, self.images[idx])
        mask_path = path.join(self.masks_path, self.masks[idx])

        img = Image.open(img_path).convert("RGB")
        mask = np.array(Image.open(mask_path))

        obj_ids = np.unique(mask)[1:]   # 0번째 품번 값은 배경의 그레이스케일 값이므로 생략

        mask = (mask == mask[:, None, None])

        boxes = []
        for i in range(len(obj_ids)):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])

            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((len(obj_ids)), )